In [ ]:
# using Pkg
# Pkg.add(url = "https://github.com/JuliaReliab/JuliaMBD.git")
using Pkg
Pkg.update()

In [ ]:
# using Pkg
# Pkg.add("DifferentialEquations")
# Pkg.add("Plots")
using DifferentialEquations
using Plots

In [ ]:
using JuliaMBD
import JuliaMBD: expr, expr_initial, get_inports, get_outports

In [ ]:
module DCMotorDiskModule

using JuliaMBD
import JuliaMBD: expr, expr_initial, get_inports, get_outports

@model DCMotorDisk begin
    ## モータ特性
    @parameter begin
        R::Float64     # 電機抵抗
        L::Float64     # インダクタンス
        K_e::Float64   # 逆起電力定数
        K_tau::Float64 # トルク定数
        J_M::Float64   # 慣性モーメント
    end
        
    ## ディスク特性
    @parameter begin
        J_I::Float64 # 慣性モーメント
        D::Float64   # 粘性減衰係数
    end
    
    @block begin
        in1 = InBlock(:v_M) # 印加電圧
        int = Integrator(:s1, outport=OutPort(:i_M))
        int1 = Integrator(:s2, outport=OutPort(:omega))
        gain = Gain(K=:(1/L))
        gain1 = Gain(K=:K_tau)
        gain2 = Gain(K=:(1/(J_M+J_I)))
        gain3 = Gain(K=:D)
        gain4 = Gain(K=:K_e)
        gain5 = Gain(K=:R)
        out1 = OutBlock(:out1)
        out2 = OutBlock(:out2)
        sum1 = Add(inports=[InPort(), InPort(), InPort()], signs=[:+, :-, :-])
        sub = Add(inports=[InPort(), InPort()], signs=[:+, :-])
    end
    
    @scope begin
        int
        int1
    end

    in1 => sum1.inports[1]
    gain4 => sum1.inports[2]
    gain5 => sum1.inports[3]
    sum1 => gain => int => [gain1, gain5, out2]
    gain1 => sub.inports[1]
    gain3 => sub.inports[2]
    sub => gain2 => int1 => [gain3, gain4, out1]
end
end

In [ ]:
module TestDCMotorDiskModule

using JuliaMBD
import JuliaMBD: expr, expr_initial, get_inports, get_outports

import Main.DCMotorDiskModule: DCMotorDisk, DCMotorDiskFunction, DCMotorDiskInitialFunction

@model TestDCMotorDisk begin
    @parameter begin
        R::Float64 = 5.7
        L::Float64 = 0.2
        K_e::Float64 = 7.16e-2
        K_tau::Float64 = 7.2e-2
        J_M::Float64 = 0.11e-3
        J_I::Float64 = 1.3e-3
        D::Float64 = 6.0e-5
        v0::Float64 = 24.0
    end

    @block begin
        motor_and_disk = DCMotorDisk()
        step = Step(steptime = 1, finalvalue = :v0, outport=OutPort(:v_M))
    end
    
    @scope step 
    
    step => motor_and_disk.v_M
end

end

In [ ]:
m = Main.TestDCMotorDiskModule.TestDCMotorDisk()

In [ ]:
@time results = simulate(m, (0.0, 10.0));

In [ ]:
results.graph